In [6]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from collections import defaultdict
import os
import pickle

from code.algorithm_alpha import algorithm_alpha
from code.algorithm_t import algorithm_t

sim = True

In [7]:
def compute_order(knots):
    if knots.ndim > 1:
        return np.flip(np.argsort(knots[:, 0]))
    else:
        return np.flip(np.argsort(knots))


def compute_out_groups(group):  # number of identified clusters
    groups_out = len(np.unique(group))
    return groups_out


algorithm_map = {
    'alpha': algorithm_alpha,
    't': algorithm_t
}


def check_algorithm(ran_var, ran_int, n_fix, sim, a_type, tol, model, n):
    all_knots = []
    all_par = []
    all_W = []
    all_hess_ran = []
    all_hess_fix = []
    all_group = []
    all_out_groups = []

    i = 0
    while i < n:
        try:
            print(i)
            knots, par, W, hess_ran, hess_fix = algorithm_map[a_type](ran_var, ran_int, n_fix, sim, tol, model, 
                                                                      fix=None, lengths=None, y=None, N=None)

            x = compute_order(knots)
            knots = knots[x]
            W = W[:, x]
            group = np.array([np.nan if np.sum(W[j, :]) == 0 else np.argmax(W[j, :]) for j in range(10)])
            out_groups = compute_out_groups(group)

            all_knots.append(knots)
            all_par.append(par)
            all_W.append(W)
            all_hess_ran.append(hess_ran)
            all_hess_fix.append(hess_fix)
            all_group.append(group)
            all_out_groups.append(out_groups)

            i = i + 1
        except:
            print('an error occurred')
            pass

    # save results
    # file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + a_type + '_' + str(tol) + '_' + model + '_' + str(
    #    n) + '.pickle'
    # f = open('output/' + file_name, 'wb')
    # pickle.dump([all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups], f)
    # f.close()

    return all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups

In [8]:
ran_var = False
ran_int = True
n_fix = 1
a_type = 't' # set 't' or 'alpha'
tol = 0.5
model = 'P'  # set 'B' or 'P'
n = 1

all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups = \
    check_algorithm(ran_var, ran_int, n_fix, sim, a_type, tol, model, n)

# save results
file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + a_type + '_' + str(
    tol) + '_' + model + '_' + str(n) + '.pickle'
f = open('output/simulation_study_results' + file_name, 'wb')
pickle.dump([all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups], f)
f.close()

0
